In [37]:
import pandas as pd
import numpy as np

In [2]:
with open('../data/slurm_wrapper_ce5.log') as f:
    data5 = ([line.split() for line in f])

In [3]:
ce5 = pd.DataFrame([[x[0] + ' ' + x[1], x[4], float(x[10]), int(x[13]), x[16]] 
                       for x in data5], columns=['BEGIN', 'USER', 'DURATION', 'RC', 'COMMAND'])

In [4]:
ce5['value_grp'] = (ce5.RC.diff(1) != 0).cumsum()

In [5]:
ce5 = ce5.reset_index()

In [6]:
f_index = pd.DataFrame({'Consecutive' : ce5.groupby('value_grp').size(),
                    'index' : ce5.groupby('value_grp').index.first(),
                    'dur' : ce5.groupby('value_grp').DURATION.first(),
                    'Returncode' : ce5.groupby('value_grp').RC.first()})

In [13]:
fe_index = f_index[(f_index['Returncode'] == 1) & (f_index['dur'] > 15)]

In [16]:
fe_df = ce5.join(fe_index[['index','Consecutive']].set_index('index'), how='inner').sort_values('BEGIN', ascending=False)

In [18]:
fe_df['COMMAND'].value_counts()

['/usr/bin/scontrol',    34694
['/usr/bin/sbatch',       7545
['/usr/bin/squeue',       3548
Name: COMMAND, dtype: int64

In [29]:
ce5[ce5['COMMAND'].str.contains('sacct')].index

Int64Index([    125,     360,     362,     428,     431,     439,     440,
                441,     442,     453,
            ...
            4770411, 4770449, 4770529, 4770562, 4770650, 4770690, 4770749,
            4770782, 4770855, 4770884],
           dtype='int64', length=357383)

In [47]:
sindex = np.array(ce5[ce5['COMMAND'].str.contains('sacct')].index)
sindex += -1
sc = ce5.join(pd.DataFrame(index = sindex), how='inner')
sc[sc['DURATION'] > 15]

,index,BEGIN,USER,DURATION,RC,COMMAND,value_grp
9118,9118,2020-10-16 16:00:16.038701,9204,18.315583,0,"['/usr/bin/scontrol',",245
10554,10554,2020-10-16 17:22:57.794663,9219,16.793026,0,"['/usr/bin/scontrol',",285
10599,10599,2020-10-16 17:24:13.928890,9202,15.571080,0,"['/usr/bin/scontrol',",285
11740,11740,2020-10-16 18:11:04.675942,9202,15.704717,0,"['/usr/bin/scontrol',",299
12440,12440,2020-10-16 18:47:50.962718,9202,17.149708,0,"['/usr/bin/scontrol',",309
...,...,...,...,...,...,...,...
4761560,4761560,2021-10-06 15:39:18.708410,9221,38.036971,1,"['/usr/bin/squeue',",271220
4761566,4761566,2021-10-06 15:39:19.632385,9221,38.039301,1,"['/usr/bin/squeue',",271222
4761568,4761568,2021-10-06 15:39:19.714047,9221,38.042240,1,"['/usr/bin/squeue',",271224
4761573,4761573,2021-10-06 15:39:19.882645,9221,38.072341,1,"['/usr/bin/squeue',",271226


In [48]:
ce5['COMMAND'].value_counts()

['/usr/bin/scontrol',    2813268
['/usr/bin/sbatch',       910534
['/usr/bin/scancel',      428813
['/usr/bin/sacct',        357383
['/usr/bin/squeue',       260888
['/usr/bin/squeue']            2
Name: COMMAND, dtype: int64